In [12]:
import sys
import math
import import_ipynb
from Generator_MatrixVectorMultiply_SI_HR_V1_6 import MatrixVectorMultiply_SI

In [13]:
#M>=1
M = input('Enter M: ')
#N>=2
N = input('Enter N: ')
#>=1, <=M
PR = input('Enter No Of Parallel Row (empty for 1): ')
M = int(M)
if M<1:
    M = 1
N = int(N)
if N<2:
    N = 2
if not PR:
    PR = 1
else:
    PR = int(PR)
    if PR>M:
        PR = M
#HRR>=2
HRR = input('Enter HRR Upper Limit (>=2, <={}, empty for 2, not all HRR values are acceptable): '.format(N))
if not HRR:
    HRR = 2
else:
    HRR = int(HRR)
    if HRR<2:
        HRR = 2
    if HRR>N:
        HRR = N
NMult = math.ceil(N/HRR)
#print("NMult: {}".format(NMult))

while math.ceil(N/NMult) != HRR:
    HRR -= 1
print("HRR calculated value: {}".format(HRR))

#MAMCS>=1
MAMCS = input('Enter Multiply-Add Max Component Size (empty for {}): '.format(NMult))

if not MAMCS:
    MAMCS = NMult
else:
    MAMCS = int(MAMCS)
    if MAMCS>NMult:
        MAMCS = NMult
    elif MAMCS<1:
        MAMCS = 1

#AdderSize>=2
if MAMCS<NMult:
    AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')

Enable_Input_Latch = input('Enable Input Latch? (empty or 0 to disable, 1 to enable): ')

RS=math.ceil(M/PR)

if RS>1:
    Enable_Vector_Latch = input('Enable Vector Latch? (empty or 0 to disable, 1 to enable): ')

Enable_SIM_Output_Registers = input('Enable SI Mux Output Registers? (empty or 0 to disable, 1 to enable): ')

#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH default value (empty for 10): ')

#INPUT_REG_DEPTH>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH default value (empty for 1): ')

#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH default value (empty for 1): ')

NumOfComponents = math.ceil(NMult/MAMCS)
if MAMCS==NMult:
    AdderSize = 0
else:
    if not AdderSize:
        AdderSize = 2
    else:
        AdderSize = int(AdderSize)
        if AdderSize<2:
            AdderSize = 2
        if AdderSize>NumOfComponents:
            AdderSize = NumOfComponents
            
if not Enable_Input_Latch:
    Enable_Input_Latch = 0
else:
    Enable_Input_Latch = int(Enable_Input_Latch)
if RS==1:
    Enable_Vector_Latch = 0;
else:
    if not Enable_Vector_Latch:
        Enable_Vector_Latch = 0
    else:
        Enable_Vector_Latch = int(Enable_Vector_Latch)

if not Enable_SIM_Output_Registers:
    Enable_SIM_Output_Registers = 0
else:
    Enable_SIM_Output_Registers = int(Enable_SIM_Output_Registers)
        
if not IN_WIDTH:
    IN_WIDTH = 10
else:
    IN_WIDTH = int(IN_WIDTH)
    if IN_WIDTH<1:
        IN_WIDTH = 1

if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH = 1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if INPUT_REG_DEPTH<0:
        INPUT_REG_DEPTH = 0

if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if MULT_PIPE_DEPTH<0:
        MULT_PIPE_DEPTH = 0

Enter M: 10
Enter N: 12
Enter No Of Parallel Row (empty for 1): 10
Enter HRR Upper Limit (>=2, <=12, empty for 2, not all HRR values are acceptable): 4
HRR calculated value: 4
Enter Multiply-Add Max Component Size (empty for 3): 
Enable Input Latch? (empty or 0 to disable, 1 to enable): 
Enable SI Mux Output Registers? (empty or 0 to disable, 1 to enable): 
Enter IN_WIDTH default value (empty for 10): 
Enter INPUT_REG_DEPTH default value (empty for 1): 
Enter MULT_PIPE_DEPTH default value (empty for 1): 


In [14]:
lghrr=math.ceil(math.log2(HRR))
lgN=math.ceil(math.log2(N))
parameters={ 
    "M":M,
    "N":N,
    "PR":PR,
    "HRR":HRR,
    "MAMCS":MAMCS,
    "ADDER_SIZE":AdderSize,
    "ENABLE_INPUT_LATCH":Enable_Input_Latch,
    "Enable_Vector_Latch":Enable_Vector_Latch,
    "ENABLE_SIM_OUTPUT_REGISTERS":Enable_SIM_Output_Registers,
    "INPUT_REG_DEPTH":INPUT_REG_DEPTH,
    "MULT_PIPE_DEPTH":MULT_PIPE_DEPTH,
    "IN_WIDTH":IN_WIDTH
}

IN_DATA_LENGHT=N*(PR+1)*IN_WIDTH
OUT_WIDTH=2*IN_WIDTH+lgN
OUT_DATA_LENGHT=PR*OUT_WIDTH

Print_To_File=1
Files_Location='MatrixVectorMultiply_GeneratedVerilogCodes/'
Is_Top=1
modules={}
GeneratedFileName=MatrixVectorMultiply_SI(parameters,Print_To_File,Files_Location,Is_Top,modules)
Files_Location+=GeneratedFileName+"/"
wrapperModuleName=GeneratedFileName+"_AXIStream"
of=open(Files_Location+wrapperModuleName+".v", 'w+')

In [15]:
# in this cell we will use the generated file and instanciate it in our new AXI wrapper module.
print("`timescale 1ns / 1ps\n",file=of)
print("module "+wrapperModuleName,file=of)
print("#(",file=of)
print("parameter IN_DATA_LENGHT= {}, ".format(IN_DATA_LENGHT),file=of)
print("parameter OUT_DATA_LENGHT= {} ".format(OUT_DATA_LENGHT),file=of)
print(")( \n",file=of)
print("input aclk,".format(),file=of)
print("input aresetn,".format(),file=of)
print("input enable,".format(),file=of)
print("input [IN_DATA_LENGHT-1:0]s_axi_data,".format(),file=of)
print("input s_axi_valid,".format(),file=of)
print("input m_axi_ready,".format(),file=of)
print("output reg[OUT_DATA_LENGHT-1:0]m_axi_data,".format(),file=of)
print("output reg m_axi_valid,".format(),file=of)
print("output reg s_axi_ready".format(),file=of)
print(");\n".format(),file=of) 
print("reg inready;".format(),file=of)
print("reg [IN_DATA_LENGHT-1:0]indata;".format(),file=of)
print("wire outready;".format(),file=of)
print("wire earlyoutready;".format(),file=of)
print("wire [OUT_DATA_LENGHT-1:0]outdata;".format(),file=of)

In [16]:
print("////////////////instancing wrapping module///////////////////".format(),file=of)
print("{} {}(".format(GeneratedFileName,GeneratedFileName+"_inst"),file=of)
print(".clk(aclk),".format(),file=of)
print(".reset(!aresetn),".format(),file=of)
print(".enable(enable),".format(),file=of)
print(".inReady(inready),".format(),file=of)
for j in range (PR):
    for i in range (N):
        print(".AE{}R{}(indata[{}:{}]),".format( i, j, IN_DATA_LENGHT-((i+j*N)*(IN_WIDTH))-1 , IN_DATA_LENGHT-((i+j*N+1)*(IN_WIDTH)) ), file=of )
for i in range (N):    
    print(".VE{}(indata[{}:{}]),".format( i, IN_DATA_LENGHT-(PR*N+i)*(IN_WIDTH)-1 , (IN_DATA_LENGHT-(PR*N+i+1)*(IN_WIDTH)) ), file=of )
print(".E{}toE{}Ready(outready),".format(0,PR-1),file=of)
for j in range (PR):
        print(".E{}(outdata[{}:{}]),".format( j, OUT_DATA_LENGHT-(j*OUT_WIDTH+1) ,  OUT_DATA_LENGHT-((j+1)*OUT_WIDTH) ), file=of)
print(".earlyE{}toE{}Ready(earlyoutready)".format(0,PR-1),file=of)
print(");".format(),file=of)

In [17]:
#in this cell we will add Axi stream compatability codes to wrapp the generated module in the previeous cell
print("/////////////////Main body/////////////".format(),file=of)
print("always @(posedge aclk)begin".format(),file=of)
print(" if(aresetn==0)begin".format(),file=of)
print("  m_axi_data<=0;".format(),file=of)
print("  m_axi_valid<=0;".format(),file=of)
print(" end".format(),file=of)
print(" else begin".format(),file=of)
print("  if(m_axi_ready==1 && m_axi_valid==1)begin".format(),file=of)
print("   m_axi_valid<=0;".format(),file=of)
print("  end".format(),file=of)
print("  else if(outready==1)begin".format(),file=of)
print("   m_axi_valid<=1;".format(),file=of)
print("   m_axi_data<=outdata;".format(),file=of)
print("  end".format(),file=of)
print(" end".format(),file=of)
print("end".format(),file=of)
print("always @(posedge aclk)begin".format(),file=of)
print(" if(aresetn==0)begin".format(),file=of)
print("  s_axi_ready<=1;".format(),file=of)
print("  inready<=0;".format(),file=of)
print("  indata<=0;".format(),file=of)
print(" end".format(),file=of)
print(" else begin".format(),file=of)
print("  inready<=0;".format(),file=of)
print("  if(s_axi_valid==1 && s_axi_ready==1)begin".format(),file=of)
print("   s_axi_ready<=0;".format(),file=of)
print("   inready<=1;".format(),file=of)
print("   indata<= s_axi_data;".format(),file=of)
print("  end".format(),file=of)
print("  else if(m_axi_valid==1 && m_axi_ready==1)begin".format(),file=of)
print("   s_axi_ready<=1;".format(),file=of)
print("  end".format(),file=of)
print(" end".format(),file=of)
print("end".format(),file=of)
print("\nendmodule",file=of)
of.close()